In [9]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

In [2]:
train_df = pd.read_csv('../../dataset/closed_world/closedworld_train.csv')
test_df  = pd.read_csv('../../dataset/closed_world/closedworld_test.csv')

In [3]:
# feature/target 분리
X = train_df.drop(columns=["label"]).values
y = train_df["label"].values

X_test = test_df.drop(columns=["label"]).values
y_test = test_df["label"].values

# train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [4]:
# Objective function
def objective(trial):

    # ----------------------------
    # Hyperparameter Search Space
    # ----------------------------

    # hidden layer sizes
    n_layers = trial.suggest_int('n_layers', 1, 3)
    layer_sizes = []
    for i in range(n_layers):
        units = trial.suggest_int(f'n_units_l{i}', 64, 512, step=64)
        layer_sizes.append(units)

    # activation
    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # optimizer
    solver = trial.suggest_categorical("solver", ["adam", "sgd"])

    # learning rate
    lr = trial.suggest_float("learning_rate_init", 1e-4, 5e-3, log=True)

    # L2 regularization
    alpha = trial.suggest_float("alpha", 1e-5, 3e-3, log=True)

    # batch size
    batch_size = trial.suggest_categorical("batch_size", [64, 128, 256])

    # ----------------------------
    # Model
    # ----------------------------
    mlp = MLPClassifier(
        hidden_layer_sizes=tuple(layer_sizes),
        activation=activation,
        solver=solver,
        alpha=alpha,
        batch_size=batch_size,
        learning_rate='adaptive',
        learning_rate_init=lr,
        max_iter=300,
        early_stopping=True,
        validation_fraction=0.15,
        n_iter_no_change=15,
        random_state=42,
        verbose=False
    )

    # Train
    mlp.fit(X_train, y_train)

    # Validation Score
    val_pred = mlp.predict(X_val)
    val_f1 = f1_score(y_val, val_pred, average='macro')

    return val_f1

In [5]:
# Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, show_progress_bar=True)

print("\n========== OPTUNA BEST RESULT ==========")
print(f"Best Macro-F1: {study.best_value:.4f}")
print("parameters:", study.best_params)

[I 2025-11-20 13:12:28,459] A new study created in memory with name: no-name-5b502b43-058f-46a0-8857-b557e32903bb


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2025-11-20 13:13:15,899] Trial 0 finished with value: 0.6855374511972857 and parameters: {'n_layers': 2, 'n_units_l0': 448, 'n_units_l1': 512, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.0005036454423286614, 'alpha': 2.0150647114596617e-05, 'batch_size': 256}. Best is trial 0 with value: 0.6855374511972857.
[I 2025-11-20 13:14:17,316] Trial 1 finished with value: 0.6885171474918442 and parameters: {'n_layers': 2, 'n_units_l0': 192, 'n_units_l1': 448, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.00035933046287541334, 'alpha': 4.713045037984182e-05, 'batch_size': 64}. Best is trial 1 with value: 0.6885171474918442.
[I 2025-11-20 13:14:22,257] Trial 2 finished with value: 0.6020293601865889 and parameters: {'n_layers': 1, 'n_units_l0': 256, 'activation': 'relu', 'solver': 'adam', 'learning_rate_init': 0.0034244896368815277, 'alpha': 0.00017448764850582014, 'batch_size': 256}. Best is trial 1 with value: 0.6885171474918442.


/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[I 2025-11-20 13:16:18,259] Trial 3 finished with value: 0.4130668181210613 and parameters: {'n_layers': 2, 'n_units_l0': 512, 'n_units_l1': 448, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.0004694216898636592, 'alpha': 0.0003791352175858608, 'batch_size': 128}. Best is trial 1 with value: 0.6885171474918442.


...

[I 2025-11-20 13:53:40,157] Trial 58 finished with value: 0.4431003157858111 and parameters: {'n_layers': 3, 'n_units_l0': 192, 'n_units_l1': 384, 'n_units_l2': 512, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.0007296978023849981, 'alpha': 1.5638240411774105e-05, 'batch_size': 256}. Best is trial 45 with value: 0.7115171635004244.
[I 2025-11-20 13:54:06,004] Trial 59 finished with value: 0.6783596650528373 and parameters: {'n_layers': 2, 'n_units_l0': 128, 'n_units_l1': 512, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.0016312281978884124, 'alpha': 5.974145116031183e-05, 'batch_size': 256}. Best is trial 45 with value: 0.7115171635004244.
[I 2025-11-20 13:55:07,947] Trial 60 finished with value: 0.6921657444611449 and parameters: {'n_layers': 3, 'n_units_l0': 256, 'n_units_l1': 320, 'n_units_l2': 448, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.0003223226712045117, 'alpha': 3.5329992120072816e-05, 'batch_size': 256}. Best is tri

/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[I 2025-11-20 14:00:40,031] Trial 70 finished with value: 0.4803882984697644 and parameters: {'n_layers': 1, 'n_units_l0': 64, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.0018759850429416805, 'alpha': 2.3001704466135202e-05, 'batch_size': 64}. Best is trial 61 with value: 0.7135485689805584.
[I 2025-11-20 14:01:15,093] Trial 71 finished with value: 0.7017358634140338 and parameters: {'n_layers': 3, 'n_units_l0': 128, 'n_units_l1': 448, 'n_units_l2': 448, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.001136106242173526, 'alpha': 2.967311036814251e-05, 'batch_size': 128}. Best is trial 61 with value: 0.7135485689805584.
[I 2025-11-20 14:01:40,137] Trial 72 finished with value: 0.7036276407386589 and parameters: {'n_layers': 3, 'n_units_l0': 128, 'n_units_l1': 448, 'n_units_l2': 384, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.001274227340607279, 'alpha': 1.6776984506741027e-05, 'batch_size': 128}. Best is trial 61 with value: 0.713

/opt/anaconda3/envs/py3_8_11/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


[I 2025-11-20 14:09:48,464] Trial 87 finished with value: 0.6713411212612588 and parameters: {'n_layers': 3, 'n_units_l0': 192, 'n_units_l1': 384, 'n_units_l2': 448, 'activation': 'tanh', 'solver': 'sgd', 'learning_rate_init': 0.0018431173228323373, 'alpha': 5.225340011504258e-05, 'batch_size': 128}. Best is trial 61 with value: 0.7135485689805584.
[I 2025-11-20 14:10:12,241] Trial 88 finished with value: 0.71097025667879 and parameters: {'n_layers': 3, 'n_units_l0': 128, 'n_units_l1': 320, 'n_units_l2': 448, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.0008865114482684497, 'alpha': 3.3698493250880016e-05, 'batch_size': 256}. Best is trial 61 with value: 0.7135485689805584.
[I 2025-11-20 14:10:40,716] Trial 89 finished with value: 0.7019068940446699 and parameters: {'n_layers': 3, 'n_units_l0': 192, 'n_units_l1': 320, 'n_units_l2': 384, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.0008762557183930737, 'alpha': 4.083751084252556e-05, 'batch_size': 2

In [8]:
# Evaluate best model on test set
best_params = study.best_params

# hidden layer sizes 구성
layer_sizes = []
for i in range(best_params["n_layers"]):
    layer_sizes.append(best_params[f"n_units_l{i}"])

best_mlp = MLPClassifier(
    hidden_layer_sizes=tuple(layer_sizes),
    activation=best_params["activation"],
    solver=best_params["solver"],
    alpha=best_params["alpha"],
    batch_size=best_params["batch_size"],
    learning_rate='adaptive',
    learning_rate_init=best_params["learning_rate_init"],
    max_iter=300,
    early_stopping=True,
    validation_fraction=0.15,
    n_iter_no_change=15,
    random_state=42,
    verbose=True
)

best_mlp.fit(X_train, y_train)

Iteration 1, loss = 4.13247275
Validation score: 0.165414
Iteration 2, loss = 3.45151985
Validation score: 0.247494
Iteration 3, loss = 3.12462260
Validation score: 0.298872
Iteration 4, loss = 2.94637004
Validation score: 0.315789
Iteration 5, loss = 2.81770994
Validation score: 0.330827
Iteration 6, loss = 2.71786443
Validation score: 0.341479
Iteration 7, loss = 2.63436026
Validation score: 0.366541
Iteration 8, loss = 2.55678985
Validation score: 0.370927
Iteration 9, loss = 2.48133713
Validation score: 0.401003
Iteration 10, loss = 2.40614910
Validation score: 0.409774
Iteration 11, loss = 2.33221929
Validation score: 0.414787
Iteration 12, loss = 2.26820294
Validation score: 0.425439
Iteration 13, loss = 2.19598094
Validation score: 0.440476
Iteration 14, loss = 2.13267995
Validation score: 0.454261
Iteration 15, loss = 2.06720844
Validation score: 0.468045
Iteration 16, loss = 2.00681368
Validation score: 0.492481
Iteration 17, loss = 1.94954914
Validation score: 0.477444
Iterat

MLPClassifier(activation='tanh', alpha=3.154728434953602e-05, batch_size=256,
              early_stopping=True, hidden_layer_sizes=(128, 320, 448),
              learning_rate='adaptive',
              learning_rate_init=0.0003541348658851275, max_iter=300,
              n_iter_no_change=15, random_state=42, validation_fraction=0.15,
              verbose=True)

In [10]:
# Evaluate
test_pred = best_mlp.predict(X_test)
test_proba = best_mlp.predict_proba(X_test)

test_acc = accuracy_score(y_test, test_pred)

test_f1_macro = f1_score(y_test, test_pred, average='macro')
test_f1_micro = f1_score(y_test, test_pred, average='micro')
test_f1_weighted = f1_score(y_test, test_pred, average='weighted')

test_roc_auc = roc_auc_score(
    y_test, test_proba,
    multi_class='ovr'
)

test_pr_auc = average_precision_score(
    y_test, test_proba,
    average='macro'
)

print("\n===== TEST RESULTS (Best Model) =====")
print(f"Accuracy:     {test_acc:.4f}")
print(f"F1_macro:     {test_f1_macro:.4f}")
print(f"F1_micro:     {test_f1_micro:.4f}")
print(f"F1_weighted:  {test_f1_weighted:.4f}")
print(f"ROC-AUC:      {test_roc_auc:.4f}")
print(f"PR-AUC:       {test_pr_auc:.4f}")


===== TEST RESULTS (Best Model) =====
Accuracy:     0.7061
F1_macro:     0.7043
F1_micro:     0.7061
F1_weighted:  0.7043
ROC-AUC:      0.9851
PR-AUC:       0.7672
